# Deepcamp: Codelab 4

**In this tutorial we will cover**:

- Large Language Models (ever heard of ChatGPT 👀 ?)
- LangChain: a tool to allow interaction between LLMs


**Author**:
- Alessio Devoto (alessio.devoto@uniroma1.it)


**Duration**: 50 mins 


🛑 **Warning**: Make sure you have an OpenAI token.

In [ ]:
%%capture
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken

In [ ]:
# usual general imports 
import os

# LangChain 🦜⛓

[Langchain](https://python.langchain.com/en/latest/index.html) makes interaction with Large Language Models easy and intuitive.

The whole library is comprised of the following building blocks:

- **models**: a model represents an llm or llm-related service offered by an API endpoint
- **prompts**: a prompt is what we feed as input to the model
- **indexes**: an index is a set of rules to split, embed and store documents so that language models can best interact with them
- **chains**: a sequence of modular components (models, prompts, other chains ...)  combined in a particular way 

The library also offers other modules that we are not interested in today: Agents and Memory. Please refer to the [LangChain official](https://docs.langchain.com/docs/) guide for those.



## Models

Let us start by exploring a simple Langchain Model.
- Model allows us to interact with a number of LLMs providers with a uniform interface. 
- Check the list of supported LLMs [here](https://python.langchain.com/en/latest/modules/models/llms/integrations.html)

We are going to use OpenAI's API, which exposes a lot of [versions](https://platform.openai.com/docs/models/gpt-4) of ChatGPT. 

To do so, you must have an OpenAI account and generate an [access key](https://platform.openai.com/account/api-keys).  

In [ ]:
os.environ["OPENAI_API_KEY"] = "" # your api key here

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(
    model_name="text-davinci-002",    # you can pick other models but this is the best speed/effectiveness tradeoff                     
    temperature=0                     # for OpenAi, see options here: https://platform.openai.com/docs/api-reference/completions/create
    ) 


In [ ]:
# let's ask something to the language model

llm_result = llm("What should I do to learn about Deep Learning and AI ?")
print(llm_result)



There is no one-size-fits-all answer to this question, as the best way to learn about deep learning and AI will vary depending on your level of expertise and experience. However, some suggestions for how to learn about deep learning and AI include attending conferences and workshops, reading books and articles on the topic, and taking online courses.


In [ ]:
#@title Try a couple of times with a different temperature and top-k! Are you getting the same answer?

from langchain.llms import OpenAI

llm = OpenAI(
    model_name="text-davinci-002",    # you can pick other models but this is the fastest one                     
    temperature=.4                     # for OpenAi, see options here: https://platform.openai.com/docs/api-reference/completions/create
    ) 

llm_result = llm("What should I do to learn about Deep Learning and AI ?")
print(llm_result)



There are a number of ways to learn about deep learning and AI. One way is to attend conferences and workshops on the topic. Another way is to read books and articles about deep learning and AI. Finally, there are online courses available that can teach you about deep learning and AI.


The stochasticity in the answer is due to the different temperature we used. 

Large language models output *a probability for each possible token in their vocabulary* (we can regard tokens as words in this case). In the **decoding step** we decide how to use that probability to choose the next token. 

Instead of always picking the token with highest probability (greedy decoding) we can decode using a number of different approaches to perform an 'exploration' op possible outcomes (top-k, top-p, temperature).

With temperature, we are basically affecting the model's prediction [confidence](https://lukesalamone.github.io/posts/what-is-temperature/) over next tokens,

## Prompts

A prompt is a nice way to format the input before it is actually fed to the model. 

This can be useful in case we want to keep part of the prompt hidden from the user but still provide an optimal response.

Let us ask a simple question to the LLM. 

In [ ]:
llm = OpenAI(
    model_name="text-davinci-002",                        
    temperature=0                     
    ) 

In [ ]:
question =  "There are 16 balls. Half of the balls are golf balls. Half of the golf balls are blue balls. How many blue golf balls are there?"

In [ ]:
llm_results = llm(question)
print(llm_results)



There are 8 blue golf balls.


Ok, looks like it got it wrong... 🤓 Can we provide a better question, which can help the model give a [better answer](https://arxiv.org/pdf/2205.11916.pdf) ?

In [ ]:
from langchain import PromptTemplate


# question will be replaced by user's prompt
template = """
{question}
Let's think about this step by step
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)

final_prompt = prompt.format(question=question) #look how we use the parameter

print (f"Final Prompt: {final_prompt}")

print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
There are 16 balls. Half of the balls are golf balls. Half of the golf balls are blue balls. How many blue golf balls are there?
Let's think about this step by step

LLM Output: 
There are 16 balls.

There are 8 golf balls.

There are 4 blue golf balls.


#### Exercise 🏋: Make a prompt

Create a prompt for getting touritic info about a city. The user will just type the city and will get information about the most important things to do in that city

In [ ]:
# your code here

In [ ]:
#@title Peek solution

from langchain import PromptTemplate

template = """
What are the most intersting places I should visit in {city} ?
Answer shortly.
"""

prompt = PromptTemplate(
    input_variables=["city"],
    template=template,
)

final_prompt = prompt.format(city='Rome')

print (f"Final Prompt: {final_prompt}")

print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
What are the most intersting places I should visit in Rome ?
Answer shortly.

LLM Output: 
There are a few interesting places to visit in Rome, such as the Colosseum, the Vatican, and the Sistine Chapel.


## Chains

Chains allow us to chain 😃 modules and prompts in order to attain a task specific goal. 
There are several kinds of chains. We can identify two main categories:

- LLM Chains: interaction of Model, Prompt and OutputParsers
- Index-Related Chains: LLM chains to deal with Documents and External "Memory"

In addition, we have a number of chains that allow you to interact in different fashions with all the modules in the library or even your own local machine processes (See later).

### LLM Chain

A simple LLM Chain accepts two main input arguments:
1. The LLM `Model` to be used
2. The `Prompt` template to be used (if any)

You can also add an output parser at the end of the chain to edit the chain's output.

In [ ]:
from langchain import LLMChain

template = """Write a {adjective} poem about {subject} in {language}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject", "language"])
llm_chain = LLMChain(
    prompt=prompt, 
    llm=llm, 
    verbose=True)

results = llm_chain.run(adjective="sad", subject="ducks", language="slang english")
print(results)



> Entering new LLMChain chain...
Prompt after formatting:
Write a sad poem about ducks in slang english.

> Finished chain.


Ducks is some real sad birds
They just waddle around all day
And every time they quack
It just sound like they're sayin' "fuck"

Life ain't easy for no ducks
They got webbed feet and feathers
And they always look so cold
They just waddle around in the water
And every time they try to fly
They just end up fallin' down

Ducks is some real sad birds
And I can't help but feel bad
For everything they go through
They just try to live their lives
The best way they know how
And I hope one day they'll find
A place where they can be happy


In [ ]:
# Just for fun

results = llm_chain.run(adjective="sad", subject="ducks", language="italian")
print(results)



> Entering new LLMChain chain...
Prompt after formatting:
Write a sad poem about ducks in italian.

> Finished chain.


I piccoli anatroccoli

nascono nel fango

e si nutrono di insetti

fino a quando non sono pronti

per volare via

lontano dal loro nido.

Ma prima di allora

i cacciatori vengono

e uccidono i piccoli

anatroccoli

per farne il prezioso

piumaggio.

Ogni anno

i piccoli anatroccoli

muoiono

senza poter mai

volare via.


### Exercise 🏋: Bash Chain

Allocate an LLM Bash chain which chains the LM with a bash process on your local machine. Then ask the llm to create a bash script which echoes `DeepCamp is so cool!` inside the shell.


Follow the steps in the template.

In [ ]:
from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

# 1. Create the language model
llm = 

# 2. Create the chain
bash_chain = LLMBashChain(llm=)

# 3. Create prompt and run the chain
# You should be very clear about what you want the llm to do, as we are using an old version of GPT and it could misunderstand you 



In [ ]:
#@title Peek Solution 👀

from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-002", temperature=0)

text = "Please write a bash script that prints 'DeepCamp is so cool!' to the console."

bash_chain = LLMBashChain(llm=llm, verbose=True)

bash_chain.run(text) # this is reallt being executed in a bash process on your local machine! 



> Entering new LLMBashChain chain...
Please write a bash script that prints 'DeepCamp is so cool!' to the console.

```bash
echo "DeepCamp is so cool!"
```['```bash', 'echo "DeepCamp is so cool!"', '```']

Answer: DeepCamp is so cool!

> Finished chain.


'DeepCamp is so cool!\n'

Now ask the chain to create a directory named `labs_deepcamp`, run it, and check if the dir was actually created.

In [ ]:
# your code here


In [ ]:
#@title Peek Solution 👀

text = "Please write a bash script that creates a directory named 'labs_deepers'."

bash_chain = LLMBashChain(llm=llm, verbose=True)

bash_chain.run(text)



> Entering new LLMBashChain chain...
Please write a bash script that creates a directory named 'labs_deepers'.

```bash
mkdir labs_deepers
```['```bash', 'mkdir labs_deepers', '```']

Answer: 
> Finished chain.


''

You can also combine single in - single out chains with `SimpleSequentialChain`, like this: 

```
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
```

 (We'll try this later 😀)


## Indexes 

The concept of Index in LangChain is quite broad. Indexes refer to ways to structure documents so that LLMs can best interact with them.

A naive way of combining LLMs and documents would be injecting the content of a document (assuming it is a text document) into the LLM as a simple prompt. This might work for small docs, as the may possibly fit the LLM [context](https://www.theatlantic.com/technology/archive/2023/03/gpt-4-has-memory-context-window/673426/). When we deal with larger or multiple documents though, this is not feasible. 

We have four classes composing the Index module:

- **Document Loaders**: responsible for loading documents from various sources.

- **Text Splitters**: responsible for splitting text into smaller chunks.

- **VectorStores**: Databases of stored documents

- **Retrievers**: nterface for fetching relevant documents to combine with language models.


As a first step, let us download a txt document containing a famous speech by Elon Musk.

In [ ]:
!wget https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/elon_musk_speech.txt

--2023-04-26 09:47:49--  https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/elon_musk_speech.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10139 (9.9K) [text/plain]
Saving to: ‘elon_musk_speech.txt’

elon_musk_speech.tx 100%[===================>]   9.90K  --.-KB/s    in 0s      

2023-04-26 09:47:52 (65.1 MB/s) - ‘elon_musk_speech.txt’ saved [10139/10139]



We could simply read the document in plain Python but we use [document loader](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) here just to keep the code homogeneous and have some nice features LangChain gives us.

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./elon_musk_speech.txt')

documents = loader.load()

# it's just the content 
#documents[:10]

As mentioned above, a whole document might be too large to fit the model's **context window**. So we split it into smaller chunks that will be embedded in a **vector space**. To do so, we use [text splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) together with embeddings.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

# we can do this in a lot of different ways
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10, separator='\n')
texts = text_splitter.split_documents(documents)

In [ ]:
texts[:5]

[Document(page_content='Elon Musk, Magicians of the 21st Century, Caltech Commencement Address,\njune, 2012', metadata={'source': './elon_musk_speech.txt'}),
 Document(page_content="I'd like to thank you for leaving 'crazy person' out of your introduction. [Laughter].", metadata={'source': './elon_musk_speech.txt'}),
 Document(page_content="I was trying to think what's the most useful thing that I can say to be useful to you in the", metadata={'source': './elon_musk_speech.txt'}),
 Document(page_content='future. And I thought, perhaps tell the story of how I sort of came to be here. How did these', metadata={'source': './elon_musk_speech.txt'}),
 Document(page_content='things happen? Maybe there are lessons there. I often find myself wondering, how did this\nhappen.', metadata={'source': './elon_musk_speech.txt'})]

We use a [vector store](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html), which takes care of actually projecting our elements into the vector space.

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, OpenAIEmbeddings())

Let's just make a quick similarity search. Given a query, we look for the embeddings (= chunks of the intial documents) that are the most similar to the query.

In [ ]:
docs = db.similarity_search("what did Elon Musk study?", k=2)
docs

[Document(page_content='Elon Musk, Magicians of the 21st Century, Caltech Commencement Address,\njune, 2012', metadata={'source': './elon_musk_speech.txt'}),
 Document(page_content='So, I studied physics and business, because in order to do these things you need to know how', metadata={'source': './elon_musk_speech.txt'})]

Finally, we can combine the database with a language model in a Retrieval Chain.

In [ ]:
from langchain.chains import RetrievalQA

# create a chain with given llm 
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model='text-davinci-002', temperature=0), 
    chain_type="stuff",      # how should we treat the document
    retriever=db.as_retriever())

                    model was transfered to model_kwargs.
                    Please confirm that model is what you intended.


In [ ]:
query = "What did Elon Musk study?"
qa_chain.run(query)

'\n\nElon Musk studied physics and business.'

Seems pretty accurate!

In [ ]:
query = "What did Elon Musk say about DeepCamp?"
qa_chain.run(query)

" I don't know."

What if we want to add some info on the fly?

In [ ]:
db.add_texts(['DeepCamp is an amazing event held in Milan for people who want to learn about deep learning.', 'I would love to go to Deepcamp, but the organizers forgot to invite me.'])

['7fa257a0-e417-11ed-9b06-0242ac1c000c',
 '7fa25a16-e417-11ed-9b06-0242ac1c000c']

In [ ]:
from langchain.chains import RetrievalQA

# create a chain with given llm 
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model='text-davinci-002', temperature=0), 
    chain_type="stuff",      # how should we treat the document
    retriever=db.as_retriever())

                    model was transfered to model_kwargs.
                    Please confirm that model is what you intended.


In [ ]:
query = "What did he say about DeepCamp?"
qa_chain.run(query)

' He said that he would love to go to DeepCamp, but the organizers forgot to invite him.'

In [ ]:
query = "did the organizers forget anything?"
qa_chain.run(query)

'\n\nThe organizers forgot to invite the speaker to Deepcamp.'

# Final Exercise 🔥

Make a Sequential chain composed of:

- a chain that reads a document of recipes and provides the steps for a recipes as output.
- a chain that takes the recipe as input the recipes and tells the user which ingredients they should buy.

Follow the template.


In [ ]:
!wget https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/recipes.txt

--2023-04-26 09:12:40--  https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/recipes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4164 (4.1K) [text/plain]
Saving to: ‘recipes.txt’

recipes.txt         100%[===================>]   4.07K  --.-KB/s    in 0s      

2023-04-26 09:12:40 (43.5 MB/s) - ‘recipes.txt’ saved [4164/4164]



1. Load and have a look at the file
2. Split it into chunks (Is there any smart way to split it ?)


In [ ]:
#@title Peek solution 👀

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

# load doc
loader = TextLoader('./recipes.txt')
documents = loader.load()
# split
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0, separator='\n\n')
texts = text_splitter.split_documents(documents)

texts[0]

Document(page_content='1. Spaghetti Bolognese: \nThis classic Italian dish is a favorite of many. Start by heating a large pot on medium heat and adding a tablespoon of butter or olive oil. Add some minced garlic and diced onions and cook until softened. Add ground beef, breaking it up with a wooden spoon as it cooks. Once the beef is cooked, add a jar of tomato sauce, a can of diced tomatoes, a tablespoon of Italian seasoning, a teaspoon of sugar, and some freshly ground black pepper. Simmer for about 20 minutes and then add cooked spaghetti to the pot. Mix until combined and serve with grated Parmesan cheese.', metadata={'source': './recipes.txt'})

3. Create a database using OpenAIEmbeddings
4. Make a quick similarity search with 'Tacos'

In [ ]:
# your code here

In [ ]:
#@title Peek solution 👀

from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, OpenAIEmbeddings())

docs = db.similarity_search("Bolognese", k=1)
docs

[Document(page_content='1. Spaghetti Bolognese: \nThis classic Italian dish is a favorite of many. Start by heating a large pot on medium heat and adding a tablespoon of butter or olive oil. Add some minced garlic and diced onions and cook until softened. Add ground beef, breaking it up with a wooden spoon as it cooks. Once the beef is cooked, add a jar of tomato sauce, a can of diced tomatoes, a tablespoon of Italian seasoning, a teaspoon of sugar, and some freshly ground black pepper. Simmer for about 20 minutes and then add cooked spaghetti to the pot. Mix until combined and serve with grated Parmesan cheese.', metadata={'source': './recipes.txt'})]

5. Allocate the RetrievealQA chain

In [ ]:
# your code here

In [ ]:
#@title Peek solution 👀

from langchain.chains import RetrievalQA

# create a chain with given llm 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",      
    retriever=db.as_retriever(),
    verbose=True
    )


6. Create a simple LLM chain with a prompt template. Given a recipe, the chain should tell us which ingredients to buy.

In [ ]:
# your code here

In [ ]:
from langchain.chains import LLMChain

template = """Given the recipe for a dish, write a list of the ingredients I should buy.

Recipe:
{recipe}
Here are the ingredients you should buy:"""
prompt_template = PromptTemplate(input_variables=["recipe"], template=template)
ingredients_chain = LLMChain(llm=llm, prompt=prompt_template)

7. Create and run a sequential chain with the prompt: `"Can you tell me how to make spaghetti Bolognese?"`

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[qa_chain, ingredients_chain], verbose=True)

query = "Can you tell me how to make spaghetti Bolognese?"
overall_chain.run('Bolognese')



> Entering new SimpleSequentialChain chain...


> Entering new RetrievalQA chain...

> Finished chain.


Spaghetti Bolognese is a classic Italian dish that is a favorite of many. To make it, start by heating a large pot on medium heat and adding a tablespoon of butter or olive oil. Add some minced garlic and diced onions and cook until softened. Add ground beef, breaking it up with a wooden spoon as it cooks. Once the beef is cooked, add a jar of tomato sauce, a can of diced tomatoes, a tablespoon of Italian seasoning, a teaspoon of sugar, and some freshly ground black pepper. Simmer for about 20 minutes and then add cooked spaghetti to the pot. Mix until combined and serve with grated Parmesan cheese.


-Butter or olive oil
-Garlic
-Onions
-Ground beef
-Tomato sauce
-Diced tomatoes
-Italian seasoning
-Sugar
-Pepper
-Spaghetti
-Parmesan cheese

> Finished chain.


'\n\n-Butter or olive oil\n-Garlic\n-Onions\n-Ground beef\n-Tomato sauce\n-Diced tomatoes\n-Italian seasoning\n-Sugar\n-Pepper\n-Spaghetti\n-Parmesan cheese'

What if we want to add a step to translate the recipe to italian? 🤔 I'll leave that to you 😉